In [1]:
from PIL import Image 
from transformers import AutoModelForCausalLM 
from transformers import AutoProcessor 

model_id = "microsoft/Phi-3-vision-128k-instruct" 
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", trust_remote_code=True, torch_dtype="auto")
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

/home/jamon/code/not_pinball/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.42it/s]
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-vision-128k-instruct:
- image_processing_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-vision-128k-instruct:
- processing_phi3_v.py
- image_processing_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Special tokens have been added in the voc

In [31]:
import os

image_directory = './data'
images = [f"{image_directory}/{file}" for file in os.listdir(image_directory) if file.endswith(('.jpg', '.jpeg', '.png'))]

messages = [ 
    {"role": "user", "content": "<|image_1|>\nIs there a pinball machine in the image?  Answer 'PINBALL' for YES or 'NOT PINBALL' for NO.  Your entire answer should be either 'PINBALL' or 'NOT PINBALL'."}, 
] 

for image_name in images:
    image = Image.open(image_name)
    prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0") 

    generation_args = { 
        "max_new_tokens": 50, 
        "temperature": 0.0, 
        "do_sample": False, 
    } 
    generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) 

    generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 
    print(f"{image_name}: {response}")


./data/not_pinball1.jpg: NOT PINBALL
./data/not_pinball2.jpg: NOT PINBALL
./data/not_pinball3.jpg: NOT PINBALL
./data/not_pinball4.jpg: NOT PINBALL
./data/pinball1.jpg: PINBALL
./data/pinball2.jpg: PINBALL
./data/pinball3.jpg: PINBALL
./data/pinball4.jpg: PINBALL
